<h2 align="center">点击下列图标在线运行HanLP</h2>
<div align="center">
	<a href="https://colab.research.google.com/github/hankcs/HanLP/blob/doc-zh/plugins/hanlp_demo/hanlp_demo/zh/tok_mtl.ipynb" target="_blank"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>
	<a href="https://mybinder.org/v2/gh/hankcs/HanLP/doc-zh?filepath=plugins%2Fhanlp_demo%2Fhanlp_demo%2Fzh%2Ftok_mtl.ipynb" target="_blank"><img src="https://mybinder.org/badge_logo.svg" alt="Open In Binder"/></a>
</div>

## 安装

无论是Windows、Linux还是macOS，HanLP的安装只需一句话搞定：

In [ ]:
!pip install hanlp -U

## 加载模型
HanLP的工作流程是先加载模型，模型的标示符存储在`hanlp.pretrained`这个包中，按照NLP任务归类。

In [1]:
import hanlp
hanlp.pretrained.mtl.ALL # MTL多任务，具体任务见模型名称，语种见名称最后一个字段或相应语料库

{'OPEN_TOK_POS_NER_SRL_DEP_SDP_CON_ELECTRA_SMALL_ZH': 'https://file.hankcs.com/hanlp/mtl/open_tok_pos_ner_srl_dep_sdp_con_electra_small_20201223_035557.zip',
 'OPEN_TOK_POS_NER_SRL_DEP_SDP_CON_ELECTRA_BASE_ZH': 'https://file.hankcs.com/hanlp/mtl/open_tok_pos_ner_srl_dep_sdp_con_electra_base_20201223_201906.zip',
 'CLOSE_TOK_POS_NER_SRL_DEP_SDP_CON_ELECTRA_SMALL_ZH': 'https://file.hankcs.com/hanlp/mtl/close_tok_pos_ner_srl_dep_sdp_con_electra_small_20210111_124159.zip',
 'CLOSE_TOK_POS_NER_SRL_DEP_SDP_CON_ELECTRA_BASE_ZH': 'https://file.hankcs.com/hanlp/mtl/close_tok_pos_ner_srl_dep_sdp_con_electra_base_20210111_124519.zip',
 'CLOSE_TOK_POS_NER_SRL_DEP_SDP_CON_ERNIE_GRAM_ZH': 'https://file.hankcs.com/hanlp/mtl/close_tok_pos_ner_srl_dep_sdp_con_ernie_gram_base_aug_20210904_145403.zip',
 'UD_ONTONOTES_TOK_POS_LEM_FEA_NER_SRL_DEP_SDP_CON_MT5_SMALL': 'https://file.hankcs.com/hanlp/mtl/ud_ontonotes_tok_pos_lem_fea_ner_srl_dep_sdp_con_mt5_small_20210228_123458.zip',
 'UD_ONTONOTES_TOK_POS_LEM

调用`hanlp.load`进行加载，模型会自动下载到本地缓存。自然语言处理分为许多任务，分词只是最初级的一个。与其每个任务单独创建一个模型，不如利用HanLP的联合模型一次性完成多个任务：

In [2]:
HanLP = hanlp.load(hanlp.pretrained.mtl.CLOSE_TOK_POS_NER_SRL_DEP_SDP_CON_ELECTRA_BASE_ZH)

## 分词
任务越少，速度越快。如指定仅执行分词，默认细粒度：

In [3]:
HanLP('阿婆主来到北京立方庭参观自然语义科技公司。', tasks='tok').pretty_print()

阿婆主 来到 北京 立方庭 参观 自然 语义 科技 公司 。


执行粗颗粒度分词：

In [4]:
HanLP('阿婆主来到北京立方庭参观自然语义科技公司。', tasks='tok/coarse').pretty_print()

阿婆主 来到 北京立方庭 参观 自然语义科技公司 。


同时执行细粒度和粗粒度分词：

In [5]:
HanLP('阿婆主来到北京立方庭参观自然语义科技公司。', tasks='tok*')

{'tok/fine': ['阿婆主', '来到', '北京', '立方庭', '参观', '自然', '语义', '科技', '公司', '。'],
 'tok/coarse': ['阿婆主', '来到', '北京立方庭', '参观', '自然语义科技公司', '。']}

`coarse`为粗分，`fine`为细分。

#### 注意
Native API的输入单位限定为句子，需使用[多语种分句模型](https://github.com/hankcs/HanLP/blob/master/plugins/hanlp_demo/hanlp_demo/sent_split.py)或[基于规则的分句函数](https://github.com/hankcs/HanLP/blob/master/hanlp/utils/rules.py#L19)先行分句。RESTful同时支持全文、句子、已分词的句子。除此之外，RESTful和native两种API的语义设计完全一致，用户可以无缝互换。

## 自定义词典
自定义词典为分词任务的成员变量，要操作自定义词典，先获取分词任务，以细分标准为例：

In [6]:
tok = HanLP['tok/fine']
tok

自定义词典为分词任务的成员变量：

In [7]:
tok.dict_combine, tok.dict_force

(None, None)

HanLP支持合并和强制两种优先级的自定义词典，以满足不同场景的需求。

不挂词典：

In [8]:
tok.dict_force = tok.dict_combine = None
HanLP("商品和服务项目", tasks='tok/fine').pretty_print()

商品 和 服务 项目


### 强制模式
强制模式优先输出正向最长匹配到的自定义词条（慎用，详见[《自然语言处理入门》](http://nlp.hankcs.com/book.php)第二章）：

In [9]:
tok.dict_force = {'和服', '服务项目'}
HanLP("商品和服务项目", tasks='tok/fine').pretty_print()

商品 和服 务 项目


与大众的朴素认知不同，词典优先级最高未必是好事，极有可能匹配到不该分出来的自定义词语，导致歧义。自定义词语越长，越不容易发生歧义。这启发我们将强制模式拓展为强制校正功能。

强制校正原理相似，但会将匹配到的自定义词条替换为相应的分词结果:

In [10]:
tok.dict_force = {'和服务': ['和', '服务']}
HanLP("商品和服务项目", tasks='tok/fine').pretty_print()

商品 和 服务 项目


### 合并模式
合并模式的优先级低于统计模型，即`dict_combine`会在统计模型的分词结果上执行最长匹配并合并匹配到的词条。一般情况下，推荐使用该模式。

In [11]:
tok.dict_force = None
tok.dict_combine = {'和服', '服务项目'}
HanLP("商品和服务项目", tasks='tok/fine').pretty_print()

商品 和 服务项目


需要算法基础才能理解，初学者可参考[《自然语言处理入门》](http://nlp.hankcs.com/book.php)。
#### 空格单词

含有空格、制表符等（Transformer tokenizer去掉的字符）的词语需要用`tuple`的形式提供：

In [12]:
tok.dict_combine = {('iPad', 'Pro'), '2个空格'}
HanLP("如何评价iPad Pro ？iPad  Pro有2个空格", tasks='tok/fine')['tok/fine']

['如何', '评价', 'iPad Pro', '？', 'iPad  Pro', '有', '2个空格']

聪明的用户请继续阅读，`tuple`词典中的字符串其实等价于该字符串的所有可能的切分方式：

In [13]:
dict(tok.dict_combine.config["dictionary"]).keys()

dict_keys([('2', '个', '空格'), ('2', '个', '空', '格'), ('2', '个空', '格'), ('2', '个空格'), ('2个', '空', '格'), ('2个', '空格'), ('2个空格',), ('iPad', 'Pro'), ('2个空', '格')])

## 单词位置

HanLP支持输出每个单词在文本中的原始位置，以便用于搜索引擎等场景。在词法分析中，非语素字符（空格、换行、制表符等）会被剔除，此时需要额外的位置信息才能定位每个单词：

In [14]:
tok.config.output_spans = True
sent = '2021 年\nHanLPv2.1 为生产环境带来次世代最先进的多语种NLP技术。'
word_offsets = HanLP(sent, tasks='tok/fine')['tok/fine']
print(word_offsets)

[['2021 年', 0, 6], ['HanLPv2.1', 7, 16], ['为', 17, 18], ['生产', 18, 20], ['环境', 20, 22], ['带来', 22, 24], ['次', 24, 25], ['世代', 25, 27], ['最', 27, 28], ['先进', 28, 30], ['的', 30, 31], ['多', 31, 32], ['语种', 32, 34], ['NLP', 34, 37], ['技术', 37, 39], ['。', 39, 40]]


返回格式为三元组（单词，单词的起始下标，单词的终止下标），下标以字符级别计量。

In [15]:
for word, begin, end in word_offsets:
    assert word == sent[begin:end]